In [17]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
%pylab inline
import pandas as pd
import json
import seaborn
import copy
import os
from util.VisualizeDataset import VisualizeDataset
from util import util as util_custom
from util.parser import get_labels,generate_csv,generate_labels,get_sensor_values
import time
from itertools import ifilterfalse
print(os.getcwd())

Populating the interactive namespace from numpy and matplotlib
/home/cts/Desktop/ML4QS/Assignments/assignments_code_git/assignment3


In [4]:
dataset_folder = './datasets'
figures_folder = './figures'
classifier_folder = './classification'
result_dataset_path = './intermediate_datafiles/'
final_dataset_name = 'final_dataset.csv'
files = os.listdir(dataset_folder)
print(files)
csv_dataset_path = os.path.join(dataset_folder,'csv/')
if not os.path.exists(result_dataset_path):
    print('Creating result directory: ' + result_dataset_path)
    os.makedirs(result_dataset_path)
if not os.path.exists(csv_dataset_path):
    print('Creating result directory: ' + csv_dataset_path)
    os.makedirs(csv_dataset_path)
if not os.path.exists(figures_folder):
    print('Creating result directory: ' + figures_folder)
    os.makedirs(figures_folder)
if not os.path.exists(classifier_folder):
    print('Creating result directory: ' + classifier_folder)
    os.makedirs(classifier_folder)

['log_assignment2', 'log_events_assignment3', 'log_assignment3', 'csv', 'log_events_assignment2']
Creating result directory: ./classification


In [13]:
dataset = pd.read_csv(os.path.join(result_dataset_path,final_dataset_name),index_col=0)
print("Dataset size:%d instances - %d features"%(dataset.shape))
features = dataset.keys().values
copy = np.array(features)
normal_features = set()
pca_features = set()
kalman_features = set()
time_domain_features = set()
frequency_domain_features = set()
for feature in copy:
    

Dataset size:942 instances - 348 features


In [14]:
print(features)

['acc_phone_x' 'acc_phone_y' 'acc_phone_z' 'gyr_phone_x' 'gyr_phone_y'
 'gyr_phone_z' 'rotation_phone_x' 'rotation_phone_y' 'rotation_phone_z'
 'rotation_phone_theta' 'rotation_phone_phi' 'labelstairsdown'
 'labelstairsup' 'labelwalking' 'acc_phone_x_kalman' 'acc_phone_y_kalman'
 'acc_phone_z_kalman' 'gyr_phone_x_kalman' 'gyr_phone_y_kalman'
 'gyr_phone_z_kalman' 'rotation_phone_x_kalman' 'rotation_phone_y_kalman'
 'rotation_phone_z_kalman' 'rotation_phone_theta_kalman'
 'rotation_phone_phi_kalman' 'pca_1' 'pca_2' 'pca_3' 'pca_4' 'pca_5'
 'pca_6' 'acc_phone_x_temp_mean_ws_50' 'acc_phone_y_temp_mean_ws_50'
 'acc_phone_z_temp_mean_ws_50' 'gyr_phone_x_temp_mean_ws_50'
 'gyr_phone_y_temp_mean_ws_50' 'gyr_phone_z_temp_mean_ws_50'
 'rotation_phone_x_temp_mean_ws_50' 'rotation_phone_y_temp_mean_ws_50'
 'rotation_phone_z_temp_mean_ws_50' 'rotation_phone_theta_temp_mean_ws_50'
 'rotation_phone_phi_temp_mean_ws_50' 'acc_phone_x_kalman_temp_mean_ws_50'
 'acc_phone_y_kalman_temp_mean_ws_50' 'acc_p